# Microsoft Movie Analysis

**Author:** Lili Beit
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Microsoft wants to enter the movie business as a creator of original content, but doesn't know anything about movies.  

I will work off the assumption that this will be a streaming subscription service, so when mining data about existing movies, I will look at ratings and popularity rather than box office returns.

However, budget will also be an important factor, since Microsoft wants a positive ROI on its subscription service.  I will seek to advise Microsoft on how to maximize the ratio of ratings to budget.

***
Questions to answer:
* What genres of movies are correlated with high ratings? (IMDB)
* What actors are correlated with high ratings? (IMDB)
* What directors are correlated with high ratings? (IMDB)
* What genres of movies are correlated with low budgets? (TN + IMDB/TMDB)
* Does runtime affect rating? (IMDB)
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [2]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 200
%matplotlib inline

In [3]:
#import Box Office Mojo Data
bom_df = pd.read_csv('data/zippedData/bom.movie_gross.csv.gz',compression='gzip')
bom_df.info()
bom_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
#import IMDB actor/crew name data
imdb_name_df = pd.read_csv('data/zippedData/imdb.name.basics.csv.gz',compression='gzip')
imdb_name_df.info()
imdb_name_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
nconst                606648 non-null object
primary_name          606648 non-null object
birth_year            82736 non-null float64
death_year            6783 non-null float64
primary_profession    555308 non-null object
known_for_titles      576444 non-null object
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [5]:
#import IMDB title aka data
imdb_title_akas_df = pd.read_csv('data/zippedData/imdb.title.akas.csv.gz', compression='gzip')
imdb_title_akas_df.info()
imdb_title_akas_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
title_id             331703 non-null object
ordering             331703 non-null int64
title                331703 non-null object
region               278410 non-null object
language             41715 non-null object
types                168447 non-null object
attributes           14925 non-null object
is_original_title    331678 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


In [6]:
#import IMDB title/basic info data
imdb_title_basics_df = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz', compression='gzip')
imdb_title_basics_df.info()
imdb_title_basics_df.head()
#looks like apostrophes print correctly

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [7]:
#import IMDB title/crew data
#not necessary for this analysis
imdb_title_crew_df = pd.read_csv('data/zippedData/imdb.title.crew.csv.gz', compression='gzip')
imdb_title_crew_df.info()
imdb_title_crew_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
tconst       146144 non-null object
directors    140417 non-null object
writers      110261 non-null object
dtypes: object(3)
memory usage: 3.3+ MB


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [8]:
#import IMDB principals data
imdb_title_principals_df = pd.read_csv('data/zippedData/imdb.title.principals.csv.gz', compression='gzip')
imdb_title_principals_df.info()
imdb_title_principals_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
tconst        1028186 non-null object
ordering      1028186 non-null int64
nconst        1028186 non-null object
category      1028186 non-null object
job           177684 non-null object
characters    393360 non-null object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [9]:
#import IMDB title ratings data
imdb_title_ratings_df = pd.read_csv('data/zippedData/imdb.title.ratings.csv.gz', compression='gzip')
imdb_title_ratings_df.info()
imdb_title_ratings_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
tconst           73856 non-null object
averagerating    73856 non-null float64
numvotes         73856 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [10]:
#import Rotten Tomatoes movie data
#not necessary for this analysis
rt_info_df = pd.read_csv('data/zippedData/rt.movie_info.tsv.gz',delimiter = '\t',encoding='latin1', 
                         compression='gzip')
rt_info_df.info()
rt_info_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
id              1560 non-null int64
synopsis        1498 non-null object
rating          1557 non-null object
genre           1552 non-null object
director        1361 non-null object
writer          1111 non-null object
theater_date    1201 non-null object
dvd_date        1201 non-null object
currency        340 non-null object
box_office      340 non-null object
runtime         1530 non-null object
studio          494 non-null object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [11]:
#import Rotten Tomatoes reviews
#not necessary for this analysis
#this data has an issue with delimiters which is throwing this error:
#  ParserError: Error tokenizing data. C error
#Whew!  The below code works.  Encoding is latin1 and need to specify tab instead of comma delimiter
rt_reviews_df = pd.read_csv('data/zippedData/rt.reviews.tsv.gz',delimiter = '\t',encoding='latin1', compression='gzip')
rt_reviews_df.info()
rt_reviews_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
id            54432 non-null int64
review        48869 non-null object
rating        40915 non-null object
fresh         54432 non-null object
critic        51710 non-null object
top_critic    54432 non-null int64
publisher     54123 non-null object
date          54432 non-null object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [12]:
#import The Movie Database (TMDb) data
tmdb_df = pd.read_csv('data/zippedData/tmdb.movies.csv.gz')
tmdb_df.info()
tmdb_df.head()

#print(tmdb_df.loc[(tmdb_df['title']).str.contains("'")]) #checking to see if apostrophes are there. Yes they are

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
Unnamed: 0           26517 non-null int64
genre_ids            26517 non-null object
id                   26517 non-null int64
original_language    26517 non-null object
original_title       26517 non-null object
popularity           26517 non-null float64
release_date         26517 non-null object
title                26517 non-null object
vote_average         26517 non-null float64
vote_count           26517 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [13]:
#import TN budget data

tn_budgets_df = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz', compression='gzip')
tn_budgets_df.info()
tn_budgets_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [28]:
# create ratings vs runtime df
runtime_ratings_df = imdb_title_basics_df.merge(imdb_title_ratings_df,how='left', on='tconst')
runtime_ratings_df.drop(columns='primary_title', inplace=True)

# drop rows with no ratings or runtime
runtime_ratings_df.dropna(subset=['averagerating','runtime_minutes'], inplace=True)

#drop rows with number of votes < 100
runtime_ratings_df = runtime_ratings_df.loc[runtime_ratings_df['numvotes'] >= 100]

#add Popularity column (product of averagerating and numvotes)
runtime_ratings_df['popularity'] = (runtime_ratings_df['averagerating'])*runtime_ratings_df['numvotes']

runtime_ratings_df.info()
# runtime_ratings_df.describe()
runtime_ratings_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27978 entries, 2 to 146122
Data columns (total 8 columns):
tconst             27978 non-null object
original_title     27978 non-null object
start_year         27978 non-null int64
runtime_minutes    27978 non-null float64
genres             27965 non-null object
averagerating      27978 non-null float64
numvotes           27978 non-null float64
popularity         27978 non-null float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.9+ MB


,tconst,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,popularity
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,31167.3
4,tt0100275,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,773.5
7,tt0137204,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263.0,2130.3
10,tt0146592,Pál Adrienn,2010,136.0,Drama,6.8,451.0,3066.8
13,tt0162942,A zöld sárkány gyermekei,2010,89.0,Drama,6.9,120.0,828.0


In [112]:
#clean up tmdb data

#drop unnecessary columns
tmdb_df = tmdb_df.drop(columns = ['Unnamed: 0','original_title','original_language','id'])

tmdb_df

,genre_ids,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...
26512,"[27, 18]",0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",0.600,2018-06-22,Trailer Made,0.0,1


In [113]:
#explore tmdb data

tmdb_df.describe() # shows that most movies have fewer than 28 votes
tmdb_df.quantile(0.86) # only around 14% of movies have more than 100 votes
tmdb_df.head()

#db is currently ordered by popularity
#popularity column seems to be a combo of vote_average and vote_count, but not sure what the formula is


,genre_ids,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27.920,2010-07-16,Inception,8.3,22186


How TMDB calculates Popularity metric for movies:

Number of votes for the day
Number of views for the day
Number of users who marked it as a "favourite" for the day
Number of users who added it to their "watchlist" for the day
Release date
Number of total votes
Previous days score

In [114]:
#clean up tmdb data - drop rows with fewer than 100 votes

tmdb_df = tmdb_df.loc[tmdb_df['vote_count'] >= 100] 
tmdb_df.describe() #only 3686 movies remain

,popularity,vote_average,vote_count
count,3686.000000,3686.000000,3686.000000
mean,11.281023,6.389392,1325.535269
std,6.196958,0.862305,2270.643058
min,0.600000,2.000000,100.000000
25%,7.557750,5.800000,186.000000
50%,9.395000,6.400000,422.000000
75%,12.894000,7.000000,1306.000000
max,80.773000,8.600000,22186.000000


In [115]:
#clean up tmdb data - investigate dups
#(I also did a bit of dup investigation before removing rows with fewer than 100 votes.
#It didn't look like the same movies and votes counted differernt places - lines were exactly alike or not same movie)

tmdb_df.duplicated(keep = False).sum() # there are 615 dup rows
# tmdb_df['title'].duplicated(keep = False).sum() #there are 645 dup titles

#let's drop the dup rows since they are clearly the same movie
tmdb_df = tmdb_df.drop_duplicates(keep = 'first')

tmdb_df['title'].duplicated(keep = False).sum() #still 33 dup titles

tmdb_df['dup_title'] = tmdb_df['title'].duplicated(keep=False) # add a column showing whether title is a dup

#investigate remaining dup titles
tmdb_df_dups = (tmdb_df.loc[tmdb_df['dup_title'] == True]).sort_values(by=['title'])
tmdb_df_dups.head(50)
tmdb_df_dups.info()
#these are not the same movie, but must add year if joining on title.  They all have different years


<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 11370 to 14240
Data columns (total 7 columns):
genre_ids       33 non-null object
popularity      33 non-null float64
release_date    33 non-null object
title           33 non-null object
vote_average    33 non-null float64
vote_count      33 non-null int64
dup_title       33 non-null bool
dtypes: bool(1), float64(2), int64(1), object(3)
memory usage: 1.8+ KB


In [116]:
#create year column
tmdb_df['release_year'] = ((tmdb_df['release_date']).str[0:4]).astype(int)

# create title_year column for merge
tmdb_df['title_year'] = (tmdb_df['title'] + '_' + tmdb_df['release_year'].astype(str))
tmdb_df.head()

,genre_ids,popularity,release_date,title,vote_average,vote_count,dup_title,release_year,title_year
0,"[12, 14, 10751]",33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,False,2010,Harry Potter and the Deathly Hallows: Part 1_2010
1,"[14, 12, 16, 10751]",28.734,2010-03-26,How to Train Your Dragon,7.7,7610,False,2010,How to Train Your Dragon_2010
2,"[12, 28, 878]",28.515,2010-05-07,Iron Man 2,6.8,12368,False,2010,Iron Man 2_2010
3,"[16, 35, 10751]",28.005,1995-11-22,Toy Story,7.9,10174,False,1995,Toy Story_1995
4,"[28, 878, 12]",27.920,2010-07-16,Inception,8.3,22186,False,2010,Inception_2010


In [117]:
#Clean up tn_budgets_df

#looks like the data is organized in order of production budget
#the data has no null values, but that doesn't mean it doesn't have placeholders.  I don't see any at first blush
#it does look like there is a problem with apostrophes.  They render as funny characters
#example: # Pirates of the Caribbean: Dead Manâs Chest
#some foreign characters also don't render correctly, like the c in fiance
# the id column is weird, it repeats every 100
#so to clean, take movies from last 20 years, nix id column, and find a way to replace apostrophes?

#first, make a year column.  currently the date is a string.  then, restrict year to >= 2000
tn_budgets_df['release_year'] = ((tn_budgets_df['release_date']).str.slice(start=-4)).astype(int) #create release_year column
tn_budgets_df = tn_budgets_df.loc[tn_budgets_df['release_year'] >= 2000] # change df to only include movies from 2000 on

tn_budgets_df.drop(columns = ['id'], inplace=True) #drop 'id' column

tn_budgets_df['movie'].replace(to_replace='â', value = "'") #replace with regular apostrophes.  It doesn't work, the weird characters don't get replaced
# print(tn_budgets_df.loc[(tn_budgets_df['movie']).str.contains("'")]) #checking to see if apostrophes are there
# print(tn_budgets_df.loc[(tn_budgets_df['movie']).str.contains('â')]) # but movies with the weird apostrophe are still here too!
#maybe it's not a big deal - there are only 40 movies with the weird character
#checked to see if compression=gzip fixed the apostrophe issue, it does not

#reset index and drop old index column:
tn_budgets_df.reset_index(inplace=True) 
tn_budgets_df.drop(columns = ['index'], inplace=True)

#cast numeric columns as int
tn_budgets_df['production_budget'] = tn_budgets_df['production_budget'].str.replace('$','').str.replace(',','').astype(int)

tn_budgets_df.info()
# tn_budgets_df.describe()
# tn_budgets_df.head()
# tn_budgets_df[500:550]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4387 entries, 0 to 4386
Data columns (total 6 columns):
release_date         4387 non-null object
movie                4387 non-null object
production_budget    4387 non-null int64
domestic_gross       4387 non-null object
worldwide_gross      4387 non-null object
release_year         4387 non-null int64
dtypes: int64(2), object(4)
memory usage: 205.8+ KB


In [91]:
#function to cast numeric columns as int.  #but it didn't work.  why?
def convert_to_int(column):
    column = column.str.replace('$','').str.replace(',','').astype(int)

In [92]:
#testing function.  did not work
convert_to_int(tn_budgets_df['domestic_gross'])
convert_to_int(tn_budgets_df['worldwide_gross'])

In [93]:
#convert these columns anyway
tn_budgets_df['worldwide_gross'] = tn_budgets_df['worldwide_gross'].str.replace('$','').str.replace(',','').astype(int)
tn_budgets_df['domestic_gross'] = tn_budgets_df['domestic_gross'].str.replace('$','').str.replace(',','').astype(int)

In [94]:
tn_budgets_df.info()
# tn_budgets_df.describe()
# tn_budgets_df.head(50)
# tn_budgets_df[500:550]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4387 entries, 0 to 4386
Data columns (total 6 columns):
release_date         4387 non-null object
movie                4387 non-null object
production_budget    4387 non-null int64
domestic_gross       4387 non-null int64
worldwide_gross      4387 non-null int64
release_year         4387 non-null int64
dtypes: int64(4), object(2)
memory usage: 205.8+ KB


In [95]:
#checking tn_budgets_df for duplicates

tn_budgets_df.duplicated().sum() # no duplicate rows
tn_budgets_df['movie'].duplicated(keep = False).sum() # only 45 duplicate titles

#I should write a function for all the below, with inputs df and column name
tn_budgets_df['dup_title'] = tn_budgets_df['movie'].duplicated(keep=False) # add a column showing whether title is a dup

#investigate remaining dup titles
tn_budgets_df_dups = (tn_budgets_df.loc[tn_budgets_df['dup_title'] == True]).sort_values(by=['movie'])
tn_budgets_df_dups
# tn_budgets_df_dups.info()
#most are not the same movie, but must add year if joining on title.  
#The only one with the same year is "Home", possibly this is the same movie

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,dup_title
2577,"Aug 14, 2015",Brothers,13000000,656688,17856688,2015,True
1715,"Dec 4, 2009",Brothers,26000000,28544157,45043870,2009,True
2524,"Dec 31, 2008",Ca$h,13700000,0,11738256,2008,True
3126,"Mar 26, 2010",Ca$h,7000000,46488,46488,2010,True
4160,"Jun 23, 2015",Crossroads,500000,0,0,2015,True
2615,"Feb 15, 2002",Crossroads,12000000,37188667,57000000,2002,True
2099,"Aug 17, 2007",Death at a Funeral,20000000,8580428,46790428,2007,True
1956,"Apr 16, 2010",Death at a Funeral,21000000,42739347,48977233,2010,True
287,"Aug 7, 2015",Fantastic Four,120000000,56117548,167849187,2015,True
450,"Jul 8, 2005",Fantastic Four,87500000,154696080,333132750,2005,True


In [96]:
#clean up tn_budgets_df data - add a column for joining
tn_budgets_df['title_year'] = (tn_budgets_df['movie'] + '_' + tn_budgets_df['release_year'].astype(str))
tn_budgets_df.head(200)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,dup_title,title_year
0,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009,False,Avatar_2009
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,False,Pirates of the Caribbean: On Stranger Tides_2011
2,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019,False,Dark Phoenix_2019
3,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015,False,Avengers: Age of Ultron_2015
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,False,Star Wars Ep. VIII: The Last Jedi_2017
5,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2015,False,Star Wars Ep. VII: The Force Awakens_2015
6,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,2018,False,Avengers: Infinity War_2018
7,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425,2007,False,Pirates of the Caribbean: At Worldâs End_2007
8,"Nov 17, 2017",Justice League,300000000,229024295,655945209,2017,False,Justice League_2017
9,"Nov 6, 2015",Spectre,300000000,200074175,879620923,2015,False,Spectre_2015


In [98]:
#join budgets table to TMDB, to get TMDB ratings

merge_df = pd.merge(tn_budgets_df, tmdb_df, left_on=  ['title_year'],
                   right_on= ['title_year'], 
                   how = 'inner')

# merge_df.info()
merge_df.tail(200)

,release_date_x,movie,production_budget,domestic_gross,worldwide_gross,release_year_x,dup_title_x,title_year,genre_ids,popularity,release_date_y,title,vote_average,vote_count,dup_title_y,release_year_y
1121,"Nov 30, 2018",The Possession of Hannah Grace,6000000,14836174,42785387,2018,False,The Possession of Hannah Grace_2018,"[27, 18]",20.263,2018-11-30,The Possession of Hannah Grace,5.1,393,False,2018
1122,"Sep 7, 2012",The Words,6000000,11494838,16369708,2012,False,The Words_2012,"[18, 53]",7.201,2012-09-07,The Words,6.6,628,False,2012
1123,"Apr 27, 2012",Bernie,6000000,9204489,10106975,2012,False,Bernie_2012,"[35, 80, 18]",7.518,2012-04-27,Bernie,6.6,503,False,2012
1124,"Jul 29, 2011",The Guard,6000000,5359774,21197454,2011,False,The Guard_2011,"[28, 35, 53, 80]",10.177,2011-07-29,The Guard,6.9,563,False,2011
1125,"Jul 18, 2014",Wish I Was Here,6000000,3591299,6591365,2014,False,Wish I Was Here_2014,"[18, 35]",7.845,2014-01-18,Wish I Was Here,6.6,440,False,2014
1126,"Mar 14, 2014",Veronica Mars,6000000,3322127,3485383,2014,False,Veronica Mars_2014,"[35, 18, 80]",7.821,2014-03-14,Veronica Mars,6.7,699,False,2014
1127,"Sep 14, 2018",Mandy,6000000,1214525,1427656,2018,False,Mandy_2018,"[28, 53, 27, 14, 9648]",16.240,2018-09-13,Mandy,6.2,618,False,2018
1128,"Dec 7, 2012",Deadfall,6000000,66351,1673635,2012,False,Deadfall_2012,"[80, 18, 53]",9.491,2012-12-07,Deadfall,5.9,357,False,2012
1129,"Jan 23, 2015",Song One,6000000,32251,437089,2015,False,Song One_2015,[18],6.206,2015-01-23,Song One,5.6,126,False,2015
1130,"May 10, 2011",The Hit List,6000000,0,0,2011,False,The Hit List_2011,"[28, 53]",6.241,2011-04-03,The Hit List,5.4,116,False,2011


Plan:
    Join data on budgets to ratings from IMDB, TMDB.  Look for movies with the lowest budget and highest ratings
    - What do they have in common?  These are my three questions:
        - Actors?
        - Run Time?
        - Genre?

Thoughts:
budget vs worldwide gross seems important.  Microsoft wants to make money right?
Let's look at movies from the past 10 years
For those that are the highest ROI, what do they have in common?
Why can't we see the title in the Rotten Tomatoes data?  Does it matter?  It does if we want to see whether
critical reviews play a part in ROI
What else plays a part?  directors, writers, actors?  composers, cinematographers?  runtime, genre?
Or maybe ROI is not important.  Maybe this is going to be a subscription service and ratings will be more important.
Microsoft is a tech company after all.  So maybe they want the highest ratings with the lowest budget.
If we are looking at ROI, start with tn_budgets_df
If we are looking at budget vs ratings, look at tn_budgets, then for ratings look at: imdb ratings, tmdb ratings.  Not sure if critic reviews (rotten tomatoes) are important here


In [25]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [26]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***